In [1]:
import json
import os
import sys
import warnings
import pickle as pkl
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve
warnings.filterwarnings("ignore")

# todo: take this out when merging with master
%cd "/home/ec2-user/SageMaker/antibody-in-pytorch/AIPT/Models/Wollacott2019"

/home/ec2-user/SageMaker/antibody-in-pytorch/AIPT/Models/Wollacott2019


In [2]:
from AIPT.Benchmarks.OAS_dataset import OAS_data_loader
from AIPT.Models.Wollacott2019 import Bi_LSTM
from AIPT.Utils.model import Model
from AIPT.Utils import loader

In [3]:
def collate_fn(batch):
    """
    Retrieve the labels for training 
    """
    return batch, [x for seq in batch for x in seq] 

# Parameters

In [4]:
para_dict = {'epoch': 4,
             'step_size': 10,
             'batch_size': 8192,
             'learning_rate': 0.01,
             'gapped': True,
             'embedding_dim': 64,
             'hidden_dim': 64,
             'fixed_len': False}
# datapath = '../../Benchmarks/OAS_dataset/data/'
datapath = 'dataloaders'

# Data Loader

In [5]:
# Training data
import random

data = pkl.load(open(os.path.join(datapath,'Human_train_seq_full_length.csv.gz'), 'rb'))
train_x = OAS_data_loader.encode_index(data=data['seq'].values, gapped=True)
train_x = random.sample(train_x, 100)
train_loader = torch.utils.data.DataLoader(train_x, collate_fn=OAS_data_loader.collate_fn)

In [8]:
print(data)
for features, label in train_loader:
    print('features')
    print(features)
    print('label')
    print(label)
    break

                                                     seq
0      --------------PGGSLRLSCAASAFTF----SAYGMHWVRQTP...
1      ---------------GGSLRLSCAASGFSF----SSYAMSWVRQAP...
2      ---------------SETLSLTCSVSGGSIS--SSAYHWSWFRQHP...
3      --------------PGGSLRLSCAASGFTF----SSFTMNWVRQAP...
4      ---------------SETLSLTCAVYGWSF----NDYYWNWIRQPP...
...                                                  ...
24995  ---------------GGSLRLSCAASGFTF----SNYWAHWVRQAP...
24996  ---------------GGSLRLSCAASGFIF----SGFAMAWVRQTP...
24997  ---------------SETLSLTCTVSGGSIS--SSSYYWGWIRQPP...
24998  --------------PGGSLRLSCVASGFPF----PTYAMMWVRQAP...
24999  ---------------GESLKISCKGSGYSF----TSYWIGWVRQMP...

[25000 rows x 1 columns]
features
[array([20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 15,  3,
       16,  9, 15,  9, 16,  1,  0, 17, 19,  5,  5, 15,  4, 20, 20, 20, 20,
       15,  5, 19, 19, 18, 15, 18,  7, 14, 13, 12, 12,  5,  8,  5,  9,  3,
       18,  7,  5,  3,  7, 11,  6, 15, 20, 20, 20,  5, 

In [10]:
# Human test data
test_data = pkl.load(open(os.path.join(datapath,'Human_test_seq_full_length.csv.gz'), 'rb'))
test_x = OAS_data_loader.encode_index(data=test_data['seq'].values)
human_test_loader = torch.utils.data.DataLoader(test_x, collate_fn=collate_fn)

# Rabbit test data
test_data = pkl.load(open(os.path.join(datapath,'Rabbit_test_seq_full_length.csv.gz'), 'rb'))
test_x = OAS_data_loader.encode_index(data=test_data['seq'].values)
rabbit_test_loader = torch.utils.data.DataLoader(test_x, collate_fn=collate_fn)

# Mouse test data
test_data = pkl.load(open(os.path.join(datapath,'Mouse_test_seq_full_length.csv.gz'), 'rb'))
test_x = OAS_data_loader.encode_index(data=test_data['seq'].values)
mouse_test_loader = torch.utils.data.DataLoader(test_x, collate_fn=collate_fn)

# Rhesus test data
test_data = pkl.load(open(os.path.join(datapath,'Rhesus_test_seq_full_length.csv.gz'), 'rb'))
test_x = OAS_data_loader.encode_index(data=test_data['seq'].values)
rhesus_test_loader = torch.utils.data.DataLoader(test_x, collate_fn=collate_fn)

KeyError: 'seq'

# Train & Test

In [ ]:
# import sagemaker

# labeling
# role = sagemaker.get_execution_role()
model = Bi_LSTM.LSTM_Bi(para_dict)
model.fit(train_loader)
output_human_train = model.predict(train_loader)

In [9]:
output_human_train

array([0.84582775, 1.24205501, 0.39678971, 0.74773982, 0.65454253,
       0.63991126, 0.54997615, 0.98607044, 0.47786674, 0.80321374,
       0.86212894, 0.76566282, 0.73376608, 0.37239635, 0.47130942,
       0.63903548, 0.65625808, 0.62880978, 1.20570134, 0.54527572,
       0.37239635, 0.30885449, 0.66469464, 0.84230634, 1.00048389,
       0.58257478, 0.66668702, 0.55106051, 1.02804318, 0.73230682,
       0.73947171, 0.70972195, 0.52299546, 1.03659278, 0.34773738,
       0.59435566, 0.57589016, 0.549156  , 0.51011231, 0.88111426,
       0.90245896, 0.49554311, 0.3683384 , 0.57589016, 0.46683016,
       0.71299301, 0.67339617, 1.15250802, 0.45569505, 0.37239635,
       0.48057578, 0.67054936, 0.52898023, 0.53807464, 0.48526446,
       0.46414264, 0.96781418, 0.67841316, 0.70076936, 1.2883675 ,
       1.01824796, 0.70961093, 0.87468957, 0.80116353, 0.5324697 ,
       0.72540295, 0.70711787, 0.42515396, 0.47097761, 0.99783455,
       0.59954442, 1.88060199, 0.59123893, 0.51722469, 0.71501

# ROC curve

In [ ]:
output_human = model.predict(human_test_loader)
plt.figure(figsize=(10,6))

# Mouse species roc curve
output_mouse = model.predict(mouse_test_loader)
label = [-1 if a < 1000 else 1 for a in range(2000)]
output = np.concatenate((output_human, output_mouse), axis=0)
mouse_fpr, mouse_tpr, _ = roc_curve(np.array(label), np.array(output))
AUC_score_mouse = roc_auc_score(np.array(label), np.array(output))
print('AUC score for mouse: ', AUC_score_mouse)
plt.plot(mouse_fpr, mouse_tpr, 'r', label='mouse(AUC=' + str(AUC_score_mouse) + ')')

# Rabbit species roc curve
output_rabbit = model.predict(rabbit_test_loader)
label = [-1 if a < 1000 else 1 for a in range(2000)]
output = np.concatenate((output_human, output_rabbit), axis=0)
rabbit_fpr, rabbit_tpr, _ = roc_curve(np.array(label), np.array(output))
AUC_score_rabbit = roc_auc_score(np.array(label), np.array(output))
print('AUC score for rabbit ', AUC_score_rabbit)
plt.plot(rabbit_fpr, rabbit_tpr, 'g', label='rabbit(AUC=' + str(AUC_score_rabbit) + ')')

# Rhesus species roc curve
output_rhesus = model.predict(rhesus_test_loader)
label = [-1 if a < 1000 else 1 for a in range(2000)]
output = np.concatenate((output_human, output_rhesus), axis=0)
rhesus_fpr, rhesus_tpr, _ = roc_curve(np.array(label), np.array(output))
AUC_score_rhesus = roc_auc_score(np.array(label), np.array(output))
print('AUC score for rhesus ', AUC_score_rhesus)
plt.plot(rhesus_fpr, rhesus_tpr, 'b', label='rhesus(AUC=' + str(AUC_score_rhesus) + ')')

plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.legend()

Check out Figure 2(b) for the comparison

# Histogram plot

In [ ]:
# Score Distribution

plt.figure(figsize=(12,8))
plt.hist(output_human_train, histtype='step', normed=True, color='red', label='Human_train')
plt.hist(output_human, histtype='step', normed=True, color='orange', label='Human_test')
plt.hist(output_rabbit, histtype='step', normed=True, color='blue', label='Rabbit')
plt.hist(output_mouse, histtype='step', normed=True, color='green', label='Mouse')
plt.hist(output_rhesus, histtype='step', normed=True, color='pink', label='Rhesus')
plt.legend()
plt.show()

Check out Figure 2(a) for the comparison